## Install and import packages

In [15]:
!pip install opensearch-py
!pip install sentence-transformers
!pip install pandas


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
  Using cached pandas-1.5.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.1 MB)
  Using cached pytz-2022.6-py2.py3-none-any.whl (498 kB)

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [35]:
import json
import pandas as pd
from opensearchpy import OpenSearch
from opensearchpy import helpers
from sentence_transformers import SentenceTransformer
transformer_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

## Connect to open search

In [10]:
# Connection variables
host = 'localhost'
port = '9200'
auth = ('admin', 'admin')

# Connect
client = OpenSearch(
    timeout = 30,
    hosts = [{'host': host, 'port': port}],
    http_compress = True, 
    http_auth = auth,
    use_ssl = False,
    verify_cers = False,
)

client.ping()

True

## Read json file and add vector column

In [18]:
# Read json as pandas dataframe
file = open('../../data/cleansing/data.json')
data = json.load(file)
df = pd.DataFrame(data)
df

,url,title,description,thumbnail,tags
0,https://www.youtube.com/watch?v=2vnliMZmwZ4,The Truth About Becoming a Web Developer,Learn how to code the right way here Master Re...,https://i.ytimg.com/vi/2vnliMZmwZ4/maxresdefau...,"[learn, code, life, of, web, developer, ..."
1,https://www.youtube.com/watch?v=O9AALzeSnto,1 Year of Learning Web Development Self Taught,This is my journey of learning Front End Web D...,https://i.ytimg.com/vi/O9AALzeSnto/maxresdefau...,"[web development, html, javascript, front e..."
2,https://www.youtube.com/watch?v=s6dMWzZKjTs,Stop wasting time when you 39 re learning to c...,My course Responsive Design for Beginners Bec...,https://i.ytimg.com/vi/s6dMWzZKjTs/maxresdefau...,[web development]
3,https://www.youtube.com/watch?v=3UK8ARg_xm4,Introduction to Web Development How to become ...,Explore Our Free Courses Learn Web Developmen...,https://i.ytimg.com/vi/3UK8ARg_xm4/maxresdefau...,"[Web Development (Interest), web developer sk..."
4,https://www.youtube.com/watch?v=C-EHoNfkoDM,FASTEST Way to Learn Web Development and ACTUA...,Learning HTML CSS and Javascript is not enough...,https://i.ytimg.com/vi/C-EHoNfkoDM/maxresdefau...,"[html, css, javascript, front end developer..."
...,...,...,...,...,...
986,https://www.youtube.com/watch?v=pOXmn7jP9gQ,Is Becoming a Data Scientist Hard,HBR named data scientist the sexiest job of th...,https://i.ytimg.com/vi/pOXmn7jP9gQ/maxresdefau...,"[is data science hard, data scientist, how t..."
987,https://www.youtube.com/watch?v=C_0mtbAWNtQ,Learn Data Science from SCRATCH with GitHub C...,Learn Data Science from SCRATCH with GitHub Co...,https://i.ytimg.com/vi/C_0mtbAWNtQ/maxresdefau...,"[vscode, vs code, data science, datascience..."
988,https://www.youtube.com/watch?v=ThS4juptJjQ,Joel Grus Learning Data Science Using Function...,PyData Seattle 2015 Everyone has an opinion on...,https://i.ytimg.com/vi/ThS4juptJjQ/maxresdefau...,"[video, compartir, teléfono con cámara, tel..."
989,https://www.youtube.com/watch?v=iROHLA_TXQM,R For Data Science Full Course Data Science Wi...,Enroll For Simplilearn s Data Science Job Gua...,https://i.ytimg.com/vi/iROHLA_TXQM/maxresdefau...,"[r for data science full course, data science..."


In [20]:
# Add the vector column
df_bk = df.copy()
df_bk = df_bk.assign(vector="")
df_bk

,url,title,description,thumbnail,tags,vector
0,https://www.youtube.com/watch?v=2vnliMZmwZ4,The Truth About Becoming a Web Developer,Learn how to code the right way here Master Re...,https://i.ytimg.com/vi/2vnliMZmwZ4/maxresdefau...,"[learn, code, life, of, web, developer, ...",
1,https://www.youtube.com/watch?v=O9AALzeSnto,1 Year of Learning Web Development Self Taught,This is my journey of learning Front End Web D...,https://i.ytimg.com/vi/O9AALzeSnto/maxresdefau...,"[web development, html, javascript, front e...",
2,https://www.youtube.com/watch?v=s6dMWzZKjTs,Stop wasting time when you 39 re learning to c...,My course Responsive Design for Beginners Bec...,https://i.ytimg.com/vi/s6dMWzZKjTs/maxresdefau...,[web development],
3,https://www.youtube.com/watch?v=3UK8ARg_xm4,Introduction to Web Development How to become ...,Explore Our Free Courses Learn Web Developmen...,https://i.ytimg.com/vi/3UK8ARg_xm4/maxresdefau...,"[Web Development (Interest), web developer sk...",
4,https://www.youtube.com/watch?v=C-EHoNfkoDM,FASTEST Way to Learn Web Development and ACTUA...,Learning HTML CSS and Javascript is not enough...,https://i.ytimg.com/vi/C-EHoNfkoDM/maxresdefau...,"[html, css, javascript, front end developer...",
...,...,...,...,...,...,...
986,https://www.youtube.com/watch?v=pOXmn7jP9gQ,Is Becoming a Data Scientist Hard,HBR named data scientist the sexiest job of th...,https://i.ytimg.com/vi/pOXmn7jP9gQ/maxresdefau...,"[is data science hard, data scientist, how t...",
987,https://www.youtube.com/watch?v=C_0mtbAWNtQ,Learn Data Science from SCRATCH with GitHub C...,Learn Data Science from SCRATCH with GitHub Co...,https://i.ytimg.com/vi/C_0mtbAWNtQ/maxresdefau...,"[vscode, vs code, data science, datascience...",
988,https://www.youtube.com/watch?v=ThS4juptJjQ,Joel Grus Learning Data Science Using Function...,PyData Seattle 2015 Everyone has an opinion on...,https://i.ytimg.com/vi/ThS4juptJjQ/maxresdefau...,"[video, compartir, teléfono con cámara, tel...",
989,https://www.youtube.com/watch?v=iROHLA_TXQM,R For Data Science Full Course Data Science Wi...,Enroll For Simplilearn s Data Science Job Gua...,https://i.ytimg.com/vi/iROHLA_TXQM/maxresdefau...,"[r for data science full course, data science...",


In [26]:
# Create the vector values
for index in df_bk.index:
    title = df_bk['title'][index]
    description = df_bk['description'][index]
    tags = df_bk['tags'][index]
    
    # Create a single string with all the text columns included
    bundle = title + ' ' + description
    
    for tag in tags:
        bundle += ' ' + tag
    
    # Transform the single string into vector
    vector = transformer_model.encode(bundle)
    df_bk['vector'][index] = vector
    
df_bk

,url,title,description,thumbnail,tags,vector
0,https://www.youtube.com/watch?v=2vnliMZmwZ4,The Truth About Becoming a Web Developer,Learn how to code the right way here Master Re...,https://i.ytimg.com/vi/2vnliMZmwZ4/maxresdefau...,"[learn, code, life, of, web, developer, ...","[-0.03570128, -0.0024455048, -0.107585415, 0.0..."
1,https://www.youtube.com/watch?v=O9AALzeSnto,1 Year of Learning Web Development Self Taught,This is my journey of learning Front End Web D...,https://i.ytimg.com/vi/O9AALzeSnto/maxresdefau...,"[web development, html, javascript, front e...","[-0.074325755, -0.23715498, 0.17783314, 0.0067..."
2,https://www.youtube.com/watch?v=s6dMWzZKjTs,Stop wasting time when you 39 re learning to c...,My course Responsive Design for Beginners Bec...,https://i.ytimg.com/vi/s6dMWzZKjTs/maxresdefau...,[web development],"[0.06282717, 0.07787557, 0.023947623, 0.040134..."
3,https://www.youtube.com/watch?v=3UK8ARg_xm4,Introduction to Web Development How to become ...,Explore Our Free Courses Learn Web Developmen...,https://i.ytimg.com/vi/3UK8ARg_xm4/maxresdefau...,"[Web Development (Interest), web developer sk...","[0.024053795, 0.02958916, -0.045379553, -0.000..."
4,https://www.youtube.com/watch?v=C-EHoNfkoDM,FASTEST Way to Learn Web Development and ACTUA...,Learning HTML CSS and Javascript is not enough...,https://i.ytimg.com/vi/C-EHoNfkoDM/maxresdefau...,"[html, css, javascript, front end developer...","[0.025534734, -0.08855252, 0.21870342, 0.01680..."
...,...,...,...,...,...,...
986,https://www.youtube.com/watch?v=pOXmn7jP9gQ,Is Becoming a Data Scientist Hard,HBR named data scientist the sexiest job of th...,https://i.ytimg.com/vi/pOXmn7jP9gQ/maxresdefau...,"[is data science hard, data scientist, how t...","[0.20442167, 0.2232194, -0.030929465, -0.10377..."
987,https://www.youtube.com/watch?v=C_0mtbAWNtQ,Learn Data Science from SCRATCH with GitHub C...,Learn Data Science from SCRATCH with GitHub Co...,https://i.ytimg.com/vi/C_0mtbAWNtQ/maxresdefau...,"[vscode, vs code, data science, datascience...","[0.010920906, -0.051748104, 0.0014827903, -0.0..."
988,https://www.youtube.com/watch?v=ThS4juptJjQ,Joel Grus Learning Data Science Using Function...,PyData Seattle 2015 Everyone has an opinion on...,https://i.ytimg.com/vi/ThS4juptJjQ/maxresdefau...,"[video, compartir, teléfono con cámara, tel...","[-0.09241347, 0.018445034, -0.18979788, -0.096..."
989,https://www.youtube.com/watch?v=iROHLA_TXQM,R For Data Science Full Course Data Science Wi...,Enroll For Simplilearn s Data Science Job Gua...,https://i.ytimg.com/vi/iROHLA_TXQM/maxresdefau...,"[r for data science full course, data science...","[-0.018594757, 0.021297734, -0.024166707, -0.0..."


## Create opensearch index

In [65]:
index_name = 'videos'

index_body = {
    'settings': {
        'index': {
            'number_of_shards': 20, 
            'number_of_replicas': 1,
            'knn': {
                'algo_param': {
                    # Default 512: https://opensearch.org/docs/latest/search-plugins/knn/knn-index#method-definitions
                    # Higher values lead to more accurate but slower searches.
                    'ef_search': 256, 
                    # Using during graph creation
                    'ef_construction': 256, 
                    # Bidirectional links for each element
                    'm': 4 
                }
            }
        },
        'knn': 'true'
    },
    'mappings': {
        'properties': {
            'url': {
                'type': 'text'
            },
            'thumbnail': {
                'type': 'text'
            },
            'title': {
                'type': 'text'
            }, 
            'description': {
                'type': 'text'
            },
            'tags': {
                # Text type can be used as array
                'type': 'text'
            }, 
            'vector': {
                'type': 'knn_vector', 
                'dimension': 384
            }
        }
    }
}

# Delete index if exists
if(client.indices.exists(index=index_name)):
    client.indices.delete(index=index_name)

# Create the index
reply = client.indices.create(index_name, index_body)
print(reply)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'videos'}


In [66]:
# Creating python array to bulk multile data at same time
data = []

for index in df_bk.index:
    url = df_bk['url'][index]
    thumbnail = df_bk['thumbnail'][index]
    title = df_bk['title'][index]
    description = df_bk['description'][index]
    tags = df_bk['tags'][index]
    vector = df_bk['vector'][index]
    
    data.append({'_index': index_name,
                 'url': url, 
                 'thumbnail': thumbnail, 
                 'title': title, 
                 'description': description, 
                 'tags': tags, 
                 'vector': vector})

In [67]:
# Bulk into opensearch
reply = helpers.bulk(client, data, max_retries=5)

In [77]:
query = input('Enter your query: ')
query_vector = transformer_model.encode(query)

open_search_query = {
    'size': 24, 
    # Fields than will be sended as response
    '_source': ['url', 'thumbnail', 'title', 'tags'],
    # Filter
    "query": {
        "bool": {
            'must': [
                {'knn': {
                    "vector": {
                        "vector": query_vector,
                        "k": 24
                    }
                }}
            ]
        }
    }
}

response = client.search(
    index = index_name, 
    size = 24, 
    body = open_search_query,
    request_timeout = 64
)

videos = [x['_source'] for x in response['hits']['hits']]
videos

Enter your query: Como puedo hacer algo facil para comer


[{'thumbnail': 'https://i.ytimg.com/vi/k9oV97JEvTk/maxresdefault.jpg',
  'title': '35 DELICIOUS RECIPES YOU CAN COOK UNDER 5 MINUTES',
  'url': 'https://www.youtube.com/watch?v=k9oV97JEvTk',
  'tags': ['cooking tips',
   ' eggshell uses',
   ' omelet recipe',
   ' repairing tips',
   ' egg recipe',
   ' recipes',
   ' cooking tricks',
   ' kitchen lifehacks',
   ' easy food to make at home',
   ' scrambled eggs',
   ' dinner',
   ' delicious treats',
   ' quick recipes',
   ' how to cook',
   ' delicious recipes',
   ' chocolate dessert',
   ' breakfast ideas',
   ' kitchen tips',
   ' best recipes',
   ' food hacks',
   ' easy recipes',
   ' kitchen crafts',
   ' cooking hacks',
   ' egg rolls',
   ' how to make',
   ' 5-minute recipes',
   ' kitchen hacks',
   ' pancake recipe',
   ' pasta recipe',
   ' recetas',
   ' ideas for kitchen']},
 {'thumbnail': 'https://i.ytimg.com/vi/n0pbTIDD-2s/maxresdefault.jpg',
  'title': 'Try this 39 EGG POTATO AND TOMATO 39 breakfast recipe Pwede i u